In [1]:
import pandas as pd
import string
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
JSON_KEY = 'CRED.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = JSON_KEY
scope = ['https://spreadsheets.google.com/feeds']
url_download = 'https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ/edit#gid=0'
url_upload = 'https://docs.google.com/spreadsheets/d/1y2QKVKT4YlsntXtaugQ6lzfuDXypURhU8kXdGiKx1Pg/edit'


In [3]:
def google_sheet_auth(worksheet_name,scope=scope, json_key= JSON_KEY, sheet_url= url_download):
    """
    Выполнение авторизации в google sheet при помощи gspread и
    ServiceAccountCredentials.

    Parameters:
    ----------
    scope: List of string variables,
        Describes URL of all servises of google sheet api that is necessary
        to gain access.
    json_key: String, global constant
        Contains path to .json key of ETL-TO-BIGQUERY project.
    sheet_url: String
    worksheet_name: String

    Returns:
    -------
    worksheet: gspread object
    """
    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_key, scope)
    gc = gspread.authorize(credentials)
    sheet = gc.open_by_url(sheet_url)
    worksheet = sheet.worksheet(worksheet_name)
    return worksheet

In [4]:
#download data from spreadsheets to pandas Dataframes
data = google_sheet_auth(worksheet_name = 'transactions').get_all_values()
transactions_df = pd.DataFrame(data[1:], columns=data[0])
data = google_sheet_auth(worksheet_name = 'leads').get_all_values()
leads_df = pd.DataFrame(data[1:], columns=data[0])
data = google_sheet_auth(worksheet_name = 'clients').get_all_values()
clients_df = pd.DataFrame(data[1:], columns=data[0])
data = google_sheet_auth(worksheet_name = 'managers').get_all_values()
managers_df = pd.DataFrame(data[1:], columns=data[0])

In [5]:
#rename columns
transactions_df.rename(columns={'created_at':'trans_created_at'}, inplace=True)
leads_df.rename(columns={'created_at':'lead_created_at'}, inplace=True)
clients_df.rename(columns={'created_at':'client_created_at'}, inplace=True)

In [6]:
#parse datetime
clients_df.client_created_at = pd.to_datetime(clients_df.client_created_at, errors='coerce')
leads_df.lead_created_at = pd.to_datetime(leads_df.lead_created_at, errors='coerce')
transactions_df.trans_created_at = pd.to_datetime(transactions_df.trans_created_at, errors='coerce')

In [7]:
#Metrics 
#заявок
leads_df['lead'] = 1  
#мусорных заявок
leads_df['garbage'] = (leads_df['l_client_id'] == '00000000-0000-0000-0000-000000000000').astype('int')

In [8]:
#формируем промежуточный результат
result = managers_df.merge(leads_df, left_on='manager_id', right_on='l_manager_id', how='right')
#количество покупок на одного покупателя за неделю после получения заявки
leads_and_trancastions = leads_df.merge(transactions_df, on='l_client_id', how='left')
leads_and_trancastions['num_buyes'] = ((leads_and_trancastions.trans_created_at > leads_and_trancastions.lead_created_at) & \
                 ((leads_and_trancastions.trans_created_at - leads_and_trancastions.lead_created_at) < pd.Timedelta("7 days"))).astype('int')
#update result
result = result.merge(leads_and_trancastions.groupby('lead_id').agg({'num_buyes':'sum'}), on='lead_id')       

In [9]:
#количество новых заявок (не было заявок и покупок от этого клиента раньше)
client_stats = leads_and_trancastions.groupby('l_client_id').agg({'lead_id':pd.Series.nunique, 
                               'transaction_id':pd.Series.nunique, 
                               'lead_created_at':'min', 
                               'trans_created_at':'min'}) \
                        .rename(columns={'lead_id':'num_leads', 'transaction_id':'num_trans',
                                         'lead_created_at':'first_lead_created', 'trans_created_at':'first_trans_created'}) \
                        .sort_values('num_leads', ascending=False)
result = result.merge(client_stats, on='l_client_id')
result['new_lead'] = (((result.num_trans == 0) | (result.lead_created_at <= result.first_trans_created)) & \
    (result.lead_created_at == result.first_lead_created)).astype('int') 

In [10]:
#количество заявок с возвратами новых клиентов (не было покупок от этого клиента раньше, но были заявки)
result['return_new_client'] = ((result.num_leads > 1) & 
                                (result.num_trans == 0) & 
                                (result.lead_created_at > result.first_lead_created)).astype('int')
#количество покупателей (кто купил в течение недели после заявки)
result['buyer'] = (result.num_buyes >= 1).astype('int')
#количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)
result['new_buyer'] = ((result.buyer == 1) & (result.first_trans_created > result.lead_created_at)).astype('int')

In [11]:
#доход от покупок, сделанных в течение недели после получения заявки
leads_and_trancastions[leads_and_trancastions['num_buyes'] > 0].groupby('lead_id').agg({'m_real_amount':'sum'})
result = result.merge(leads_and_trancastions[leads_and_trancastions['num_buyes'] > 0].groupby('lead_id').agg({'m_real_amount':'sum'}),
                        on='lead_id', how='left')
result['m_real_amount'].fillna(0, inplace=True)

In [12]:
#формируем дату
result['date'] = result.lead_created_at.apply(lambda x: x.isoformat().split('T')[0])

In [13]:
#clear result
columns_to_drop = ['lead_id', 'lead_created_at', 'd_utm_medium','l_client_id','num_leads', 
                    'num_trans', 'first_lead_created', 'first_trans_created']
upload = result.drop(columns_to_drop, axis=1)

In [14]:
#диапазон ячеек
ss_range = 'A1:' + string.ascii_uppercase[upload.shape[1]-1] + str(upload.shape[0] + 1)
ss_range

'A1:N3338'

In [15]:
#готовим данные для загрузки
title = list()
values_to_ss = list()
for col in upload.columns: title.append(col)
values_to_ss.append(title)
for item in range(upload.shape[0]):
        values_to_ss.append([str(upload[column][item]) for column in upload.columns])

In [16]:
#upload result
upload_worksheet = google_sheet_auth(
    scope = ['https://spreadsheets.google.com/feeds'],
    json_key = JSON_KEY,
    sheet_url = url_upload,
    worksheet_name = 'sheet1')
upload_worksheet.update(ss_range, values_to_ss)

{'spreadsheetId': '1y2QKVKT4YlsntXtaugQ6lzfuDXypURhU8kXdGiKx1Pg',
 'updatedRange': 'sheet2!A1:N3338',
 'updatedRows': 3338,
 'updatedColumns': 14,
 'updatedCells': 46732}